<a href="https://colab.research.google.com/github/khodid/2020_SAI_MONING2/blob/master/CNNLec10_6_ResNet_for_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resnet을 cifar에

적용해서 학습을 진행해보자.

colab에서 visdom을 쓰기는 어려운 것 같으니

일단은 visdom 관련 코드는 지우고 진행하겠다.

anaconda와 jupyter notebook을 예전에 받아놓은 적이 있어서 이걸 쓰면 되긴 하는데.. PyTorch 환경에서는 돌려본 적이 없어서 세팅할 것이 많다.

In [0]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim

In [19]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])

# 테스트용 데이터셋 가져오기
trainset = torchvision.datasets.CIFAR10(root='./data/cifar10', train=True, download=True, transform=transform)

train_data = trainset.data # 강의 github에 있던 train_data 부분이 현재는 그냥 'data'라는 이름인 것 같다.
print(train_data.shape)

# 계산하는 방법
train_data_mean = train_data.mean( axis=(0,1,2) )  # 각 축의 평균
train_data_std = train_data.std( axis=(0,1,2) )    # 각 축의 표준 편차


print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255                    # 평균을 255로 나눔
train_data_std = train_data_std / 255                      # 표준편차를 255로 나눔

print(train_data_mean)
print(train_data_std)


Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [5]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),     # padding=4로 둘러준 후에 랜덤하게 32만큼 뜯어오겠다는 뜻. 
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


위 까지가 데이터를 받아오는 과정이었고,

이 다음 resnet부터가 주목할 부분이다.

일단 colab 환경에서 resnet.py를 불러오는 거 자체에서 좀 고민을 했는데... 강의 일시정시 시켜놓고 한참 알아보고 겨우 해결하니깐 정지시킨 부분 바로 5초 뒤에 
```python 
import torchvision.models.restnet as resnet
```
을 알려준다... ^^.

암튼 알아보는 과정에서 resnet에 대한 분석 설명이 잘 되어있는 [문서](https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_convolutional-modern/resnet.ipynb#scrollTo=cD3nWXgM7D_C)를 하나 건졌는데, 이걸 작성한 [dive into deep github](https://github.com/d2l-ai/d2l-en)에 유용한 자료가 꽤 있다. 외국의 모딥 같은 사이트인 것 같은데...



In [0]:
import torchvision.models.resnet as resnet

In [0]:
# resnet을 'as resnet' 없이 그대로 torchvision에서 import할 경우엔 생략해도 되지 않나...? 라고 생각했는데 아니었음
conv1x1=resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock= resnet.BasicBlock

In [0]:
# 모딥 강의에서는 원래의 선언에서 변형을 가했다.
# cifar에서 사용하는 것은 resnet이 목표하던 224x224보다 훨씬 작은 32x32크기의 이미지이기 때문.
# 때문에 padding도 줄였고, maxpooling도 빼고, 레이어도 줄였다.
# size도 32x32에 맞춰서 바꾸었다.

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        #x.shape =[1, 16, 32,32]
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)

        x = self.layer1(x)
        #x.shape =[1, 128, 32,32]
        x = self.layer2(x)
        #x.shape =[1, 256, 32,32]
        x = self.layer3(x)
        #x.shape =[1, 512, 16,16]
        x = self.layer4(x)
        #x.shape =[1, 1024, 8,8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [0]:
resnet50 = ResNet(resnet.Bottleneck, [3, 4, 6, 3], 10, True).to(device) 
#1(conv1) + 9(layer1) + 12(layer2) + 18(layer3) + 9(layer4) +1(fc)= ResNet50

In [10]:
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [11]:
# 모델 테스트
a=torch.Tensor(1,3,32,32).to(device)
out = resnet50(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [0]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(resnet50.parameters(), lr = 0.001)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

Plot 선언하는 부분은 colab의 한계로 못 넣었음.

**define acc_check function**

정확도를 체크하는 부분을 복붙해서 약간 수정한 코드라고 한다.

save를 하면 model이라는 폴더에 정확도와 에폭을 저장하겠다는 의미로.

In [0]:
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save:
        torch.save(net.state_dict(), "/content/gdrive/My Drive/Colab Notebooks/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

**Training**

In [25]:
print(len(trainloader))
# 에폭은 줄이고 돌리자...
epochs = 30

for epoch in range(epochs):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        # print("finished resnet50")
        loss = criterion(outputs, labels)
        # print("finished loss")
        loss.backward()
        # print("finished backward")
        optimizer.step()
        # print("finished opt")
        # 너무 오래 걸려서 돌아가는 건지 오류가 난 건지 알 수가 없어서...
        # 한 번 출력하게 해본건데 이미지 하나 처리하는데 30초는 넘게 걸리는 것 같다.
        # 실습... 마칠 수 있을까...

        lr_sche.step() 
        # 모딥 github의 코드를 그대로 돌리니깐 
        #optimizer.step()보다 lr_sche.step()을 앞에 두지 말라는 경고가 떠서 위치를 옮겼음.

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            #value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    
    #Check Accuracy
    acc = acc_check(resnet50, testloader, epoch, save=1)
    #value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    

print('Finished Training')

[14,   150] loss: 1.784
[14,   180] loss: 1.794
Accuracy of the network on the 10000 test images: 34 %
14
[15,    30] loss: 1.786
[15,    60] loss: 1.772
[15,    90] loss: 1.784
[15,   120] loss: 1.777
[15,   150] loss: 1.780
[15,   180] loss: 1.779
Accuracy of the network on the 10000 test images: 34 %
15
[16,    30] loss: 1.785
[16,    60] loss: 1.783
[16,    90] loss: 1.776
[16,   120] loss: 1.781
[16,   150] loss: 1.778
[16,   180] loss: 1.774
Accuracy of the network on the 10000 test images: 34 %
16
[17,    30] loss: 1.785
[17,    60] loss: 1.769
[17,    90] loss: 1.777
[17,   120] loss: 1.786


KeyboardInterrupt: ignored

진행이 하도 안 돼서 colab 설정도 gpu로 바꾸어 주고

SGD가 잘 안 먹히는 것 같아서 optimizer도 Adam으로 바꾸었는데

학습상태가 많이 안 좋다...

CNN 공부를 이어가려면 구글에서 TPU나 GPU 클라우드를 월정액으로 써야 하나...?

10 epoch 진행하는데 25분 넘게 걸림. 근데 학습도 거의 안 되는 걸 봐선 뭔가 잘못 설정한게 있나 싶다.

**Model Accuracy Testing**

In [26]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 34 %
